Morgan Bauer
score_prediction.ipynb
Predicts the fantasy score of an individual NBA player for one game based on the
last 10 games using an RNN.

Import required libraries

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import os
import csv

Prepare data

In [44]:
teams = {'ATL': '0',
         'BOS': '1',
         'BKN': '2',
         'CHA': '3',
         'CHI': '4',
         'CLE': '5',
         'DAL': '6',
         'DEN': '7',
         'DET': '8',
         'GSW': '9',
         'HOU': '10',
         'IND': '11',
         'LAC': '12',
         'LAL': '13',
         'MEM': '14',
         'MIA': '15',
         'MIL': '16',
         'MIN': '17',
         'NOP': '18',
         'NYK': '19',
         'OKC': '20',
         'ORL': '21',
         'PHI': '22',
         'PHO': '23',
         'POR': '24',
         'SAC': '25',
         'SAS': '26',
         'TOR': '27',
         'UTA': '28',
         'WAS': '29'}
path = "/Users/morganbauer/Documents/GitHub/fantasy_basketball/rnn_score_prediction/training_data/foxde01"
dataset = None
#TODO: all_fan_pts
#TODO: all_raw_data
for file in os.listdir(path):
    if file[:4] == '2023' or file[:4] == '2024':
        print(file)
        file_path = f"{path}/{file}"
        with open(file_path, "r") as infile:
            reader = csv.reader(infile)
            games = []
            for row in reader:
                try:
                    row[3] = teams[row[3]]
                    for pos, elem in enumerate(row):
                        row[pos] = float(elem)
                    games.append(row)
                except KeyError:
                    pass
        fan_pts = np.zeros(len(games))
        raw_data = np.zeros((len(games), len(games[0])))
        for i, game in enumerate(games):
            fan_pts[i] = game[1]
            raw_data[i, :] = game
        

2023_foxde01.csv
[49.9  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
[[24.   49.9   1.   ...  5.   33.   35.15]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]]
[49.9 51.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
[[24.         49.9       